In [1]:
import matplotlib.pyplot as plt
import json
from pennylane import numpy as np

In [2]:
with open('exp_runs/simplified2design_3layers.json', 'r') as reader:
    # Read & print the entire file
    data = reader.read()
    s2design = json.loads(data)

with open('exp_runs/kUpCCGSD_1k.json', 'r') as reader:
    # Read & print the entire file
    data = reader.read()
    kUpCCGSD_1k = json.loads(data)

with open('exp_runs/h2_fci.json', 'r') as reader:
    # Read & print the entire file
    data = reader.read()
    fci_h2 = json.loads(data)    

with open('exp_runs/double_ex.json', 'r') as reader:
    # Read & print the entire file
    data = reader.read()
    double_ex = json.loads(data)      


s2design_adam = [energy[-1] for energy in s2design["results"]["demon_adam"]["energies"]] 
kUpCCGSD_1k_adam = [energy[-1] for energy in kUpCCGSD_1k["results"]["demon_adam"]["energies"]][:len(s2design_adam)]
double_ex_adam = [energy[-1] for energy in double_ex["results"]["demon_adam"]["energies"]][:len(s2design_adam)]

test_points=fci_h2["points"]
fci_energies = fci_h2["energies"]
